In [1]:
!pip install faiss-cpu --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 38.5 MB/s eta 0:00:00


In [2]:


! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e98bf0488c3df4ecbeba7653542c49c43f7ac42298b73de14fe6386529495fd8
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

# Dataset Initialization

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_json("/content/drive/Capstone-esya.ai/DATA/summarized_df100.json")
df

FileNotFoundError: ignored

We will take all samples from `text` and build sentence embeddings for each - which we can then store in FAISS.

In [ ]:
#select only text for futher embedding
sentences = df['text'].tolist()
sentences[:5]

In [ ]:
len(set(sentences))

Tokenized the words

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

We can save/load from file in the case of needing to reload the notebook for any reason later.

In [ ]:
sentence_embeddings.shape[0]

We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [ ]:
d = sentence_embeddings.shape[1]
d

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is the specify the vector dimensionality - which in this case is `d == 768` (to align with the sentence-BERT model output embeddings of size `768`).

In [ ]:
index = faiss.IndexFlatL2(d)

In [ ]:
index.is_trained

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

Then search given a query `xq` and number of nearest neigbors to return `k`.

In [ ]:
k = 5
xq = model.encode(["politics situation"])

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

Here we're returning text of the news article, which returns:

In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

In [ ]:
# Interactive search loop
while True:
    # Request user input for a query
    query_text = input("What do you want to search (or 'q' to quit): ")

    # Check if the user wants to quit
    if query_text.lower() == 'q':
        print("Exiting the search - thank you")
        break

    # Perform similarity search
    k = 5  # Number of nearest neighbors to retrieve
    distances, indices = index.search(xq, k)

    # Get similar texts
    similar_texts = df.iloc[indices[0]]

    # Display the similar texts
    print("\nSimilar Texts:")
    print(similar_texts)